In [3]:
import re
import json
import htmlmin
from glob import glob

In [18]:
issueID = '119804319X00920190508'

In [19]:
with open(f'../keyphrases/{issueID}.json', 'r') as r:
    data = json.load(r)

meeting = data['meetingRecord'][0]

issue_name = f'第{meeting["session"]}回国会 {meeting["nameOfHouse"]} {meeting["nameOfMeeting"]} {meeting["issue"]}'
issue_date = meeting['date']

kps = {}
num_speech = len(meeting['speechRecord'])
for i, speech in enumerate(meeting['speechRecord']):
    for kp in speech['keyphrase_full']:
        if kp not in kps:
            kps[kp] = []
        kps[kp].append(i)

In [22]:
list(kps.keys())[:100]

['拡充',
 '警察庁長官官房審議官',
 '審議官',
 '対策',
 '経済産業省',
 '大臣',
 '地震',
 '機構',
 '法務省大臣官房審議官',
 '官房',
 '選任',
 '元年',
 '令和元年',
 '警察庁',
 '経済産業省大臣官房審議官',
 '機構拡充',
 '内閣官房内閣審議官',
 '地震対策',
 '補欠選任',
 '国土交通省',
 '水曜日',
 '十分開議',
 '補欠',
 '長官',
 '観光庁',
 '観光庁審議官',
 '令和',
 '内閣',
 '十分',
 '内閣官房',
 '法務省',
 '開議',
 '提出',
 '審査',
 '異議なし',
 'なし',
 '内閣提出',
 '異議',
 '本案審査',
 '本案',
 '自民党',
 '運転',
 '中谷真一',
 'けさ大津',
 'きょう',
 '自動',
 '中谷',
 '国交省',
 'お見舞い',
 'けさ',
 '真一',
 'メリット',
 '自動運転',
 '大津',
 '自動車',
 '分野',
 '関係省庁',
 'レベル',
 '移動サービス',
 '社会課題',
 '無人自動運転移動サービス',
 '操作',
 '負担',
 '軽減',
 '移動手段',
 '大きな',
 '自家用車',
 '社会',
 '交通弱者',
 '省力化',
 '地域',
 '距離',
 '限定地域',
 '自動制御',
 '無人',
 '公共',
 '生産性向上',
 'ｉｔｓ',
 '件数',
 'ロードマップ',
 '運転者',
 '展開',
 '高齢者',
 'サービス',
 '交通',
 '実用化',
 '不足',
 '移動手段確保',
 '死亡',
 '国際競争力',
 '運転操作',
 '確保',
 '国際展開',
 '運転者不足',
 'ノウハウ',
 '物流',
 '発生',
 '具体的',
 '課題',
 '国際']

In [31]:
freq = sorted([(kp, len(kps[kp])) for kp in kps if len(kps[kp]) > 1], key=lambda x: x[1])

In [43]:
import numpy as np

distances = []
for kp in kps:
    pos = kps[kp]
    if len(pos) < 2:
        continue

    dists = [pos[i] - pos[i-1] for i in range(1, len(pos))]    
    distances.append((kp, np.mean(dists), len(pos)))

sorted(distances, key=lambda x: -x[1])

[('異議なし', 143.0, 2),
 ('異議', 143.0, 2),
 ('提出', 136.0, 2),
 ('内閣提出', 136.0, 2),
 ('最先端', 135.0, 2),
 ('原因究明', 125.0, 2),
 ('究明', 125.0, 2),
 ('セキュリティ', 121.0, 2),
 ('省力化', 117.0, 2),
 ('生産性', 117.0, 2),
 ('公明党', 117.0, 2),
 ('交流', 109.0, 2),
 ('ｉｔｓ', 106.0, 2),
 ('ロードマップ', 106.0, 2),
 ('構想', 106.0, 2),
 ('官民ｉｔｓ構想', 106.0, 2),
 ('型式指定制度', 105.0, 2),
 ('拡充', 104.0, 2),
 ('全体', 104.0, 2),
 ('無線', 104.0, 2),
 ('運営', 104.0, 2),
 ('立憲民主党', 104.0, 2),
 ('立憲', 104.0, 2),
 ('民主党', 104.0, 2),
 ('アメリカ', 103.0, 2),
 ('ターゲット', 103.0, 2),
 ('環境整備', 103.0, 2),
 ('いわゆるエーミング', 103.0, 2),
 ('作動状況', 100.0, 2),
 ('タスクフォース', 99.0, 2),
 ('なかっ', 97.0, 2),
 ('範囲', 93.0, 2),
 ('範囲拡大', 93.0, 2),
 ('自家用車', 91.0, 2),
 ('限定地域', 91.0, 2),
 ('限定', 91.0, 2),
 ('領域', 90.0, 2),
 ('安心感', 89.0, 2),
 ('事件', 85.0, 2),
 ('方針', 84.0, 2),
 ('確認', 84.0, 2),
 ('作動継続', 82.0, 2),
 ('令和', 81.0, 2),
 ('自動車整備技術', 81.0, 2),
 ('一定程度', 80.0, 2),
 ('一定', 80.0, 2),
 ('有識者', 79.0, 2),
 ('従業員', 78.0, 2),
 ('効率化', 78.0, 2),
 ('サーバー', 78.0, 